# Classe RSA

## Inicialização

Foi implementada uma classe que permite a geração de esquemas RSA. Esta recebe necessariamente um valor como argumento, responsável por determinar o parâmetro de segurança. Com este, segue-se a inicialização das variáveis da classe:
1. Geram-se dois dois primos `p` e `q` pseudoaleatórios.
2. Calcula-se o comprimento `N` das chaves do esquema a partir do produto dos primos. 
3. Calcula-se o valor da função totiente de Euler `φ(N) = (p - 1) * (q - 1)`
4. Recorrendo a um anel de inteiros `G` de tamanho `φ(N)`, gera-se a chave pública do esquema `pub ∈ [0.. φ(N) - 1]`, tendo em conta que o maior divisor comum entre esta e `φ(N)` deverá ser igual a 1.
5. A partir da chave pública, deriva-se a chave privada `priv = pub^-1 mod φ(N)`


## Cifragem e decifragem

Para cifrar uma mensagem, é necessário primeiro traduzi-la para um valor que será depois processado. Neste exemplo, após uma maiusculização das letras, é extraído o valor ASCII de cada carater da mensagem e concatenado num só valor `ptxtval`.

A mensagem é cifrada do seguinte modo:
1. Conversão da mensagem num valor `ptxtval`.
2. Cifra do valor da mensagem com a chave pública `ctxt = ptxtval^pub (mod N)`. 

A decifragem do criptograma é executada no processo inverso. Tendo em conta que o programa certifica que as letras são maiúsculas, sabe-se que cada carater se caraterizará por um inteiro de dois dígitos. Como tal, após a decifragem, será necessário separar o valor do texto limpo num conjunto de valores de dois dígitos que serão convertidos de volta ao respetivo carater da tabela ASCII.

O criptograma é decifrado do seguinte modo:
1. Decifragem do valor do criptograma recorrendo à chave privada `ptxt = crypto^priv (mod N)`
2. Conversão do valor do texto limpo de volta para a mensagem original.


## Assinatura de mensagens

O par de chaves RSA pode também ser utilizado na assinatura de mensagens. Neste caso, como não é necessário recuperar a mensagem original para verificar a assinatura, recorre-se a funções de *hash* para gerar um valor único com o qual se cria a assinatura.

A assinatura é construída da seguinte forma:
1. É gerado o valor de *hash* da mensagem a assinar
2. O valor de *hash* é convertido num inteiro `hashval`
3. Recorrendo a um anel de inteiros `G`, é calculada a assinatura utilizando a chave privada `signature = sign^priv (mod N)`

A verificação da assinatura é feita através da comparação do valor de *hash* da mensagem recebida com o valor de *hash* recebido após anulada a cifra da chave privada utilizando a chave pública.

A verificação da assinatura é efetuada da seguinte forma:
1. É gerado o valor de *hash* `hashval` da mensagem recebida
2. É calculado o valor de *hash* `signature` associado à mensagem recebida
3. São comparados os dois valores. Se forem iguais, verifica-se a assinatura.

In [1]:
from sage.cpython.string import bytes_to_str, str_to_bytes
import hashlib

class RSA:
    def __init__(self, size):

        p = random_prime(2**size-1,True,2**(size-1))
        q = random_prime(2**size-1,True,2**(size-1))
        while p == q:
            p = random_prime(2**size-1,True,2**(size-1))
            q = random_prime(2**size-1,True,2**(size-1))
        
        self.G = IntegerModRing(p*q)
        phi = (p - 1) * (q - 1)
        
        R = IntegerModRing(phi)
        self.pub = R.random_element(phi)
        while gcd(self.pub, phi) != 1:
            self.pub = R.random_element(phi)
        
        self.priv = 1/self.pub
        
        
    def encrypt(self, msg):
        pub = self.pub
        G = self.G
        ptxt = []
        
        for i in str.upper(msg):
            x = ord(i)
            ptxt.append(x)
            
        ptxtval = int(''.join(map(str, ptxt)))
        
        ctxt = G(ptxtval)**pub
        
        return ctxt
    
    
    def decrypt(self, crypto):
        priv = self.priv
        msg = ""
        
        ptxt = crypto**priv
        aux = str(ptxt)
        
        for i in xrange(0,len(aux)-1,2):
            x = chr(int(aux[i] + aux[i+1]))
            msg += x
        
        return str.lower(msg)
    
    
    def sign(self, msg):
        G = self.G
        
        hashval = int(hashlib.sha256(str(msg)).hexdigest(), 16)
        
        signature = G(hashval)**self.priv
        
        return signature
    
    
    def verify(self, crypto, sign):
        
        hashval = int(hashlib.sha256(str(crypto)).hexdigest(), 16)
        
        signature = sign**self.pub
        
        if hashval == signature:
            print 'Valid signature'
        else:
            print 'Invalid signature'

# Exemplo de utilização

In [2]:
# Initialise class and generate key pair
rsa = RSA(1024)

# Message
msg = "exemplo de uma mensagem!"

# Encrypt
ctxt = rsa.encrypt(msg)

print("ctxt = " + str(ctxt))

# Decrypt
ptxt = rsa.decrypt(ctxt)

print("\nptxt = " + ptxt)

# Sign
sign = rsa.sign(ctxt)

print("\nsign = " + str(sign) + "\n")

# Verify
rsa.verify(ctxt, sign)

ctxt = 8486856802673793274924251937017302937516324051001138450556692332062712189646927000161378799145304384353023268050519623084073057346016017251537661657037307889979731143970097241176657963327187465892549358193157879231998368086075648837852099471152622807052728197395557925267082672779256212782856270272415839199793993275101364469181952262797473624986094851809425827439388644032606241963139997643390353155405700305387372153360368353677887894407983104089029474500411247180228227502003643984221052540818880495110223603876596478701348232869312922639456369871282172906624943872021169600342439415375154870351997034526802530007

ptxt = exemplo de uma mensagem!

sign = 86822415193805951314368649618881192966181063948828256253044501358093272004994984670550634754921576761013558285246662673773974062415749533720913677265631549448969458079892074141204591431474573173331419960454821464383522362578853432955480901414789317359204748200650232482202657164384302922876598270618953612941601828280751412327798891837